In [1]:
!pip install nltk

In [9]:
import random
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
# init file
words = []
classes = []
documents = []
ignore_words = ["?", "!"]
data_file = open("/content/drive/MyDrive/Colab Notebooks/chatbot/intents.json").read()
intents = json.loads(data_file)

In [13]:
intents

{'intents': [{'tag': 'greetings',
   'patterns': ['hi there',
    'hello',
    'haroo',
    'yaw',
    'wassup',
    'hi',
    'hey',
    'holla',
    'hello'],
   'responses': ['hello thanks for checking in',
    'hi there, how can i help you'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['bye', 'good bye', 'see you later'],
   'responses': ['have a nice time, welcome back again', 'bye bye'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'okay',
    'Thank you',
    'thankyou',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me',
    'wow',
    'great'],
   'responses': ['Happy to help!',
    'Any time!',
    "you're welcome",
    'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [''],
   'responses': ["Sorry, I didn't understand you",
    'Please give me more info',
    'Not sure I understand that'],
   'context': ['']},
  {'tag': 'name1',
   'patterns': ["what's your name?", 'who are you?'],
   'resp

In [14]:
# words
for intent in intents["intents"]:
    for pattern in intent["patterns"]:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent["tag"]))

        # adding classes to our class list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])


In [19]:
# lemmatizer
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents", documents)

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)

100 documents [(['hi', 'there'], 'greetings'), (['hello'], 'greetings'), (['haroo'], 'greetings'), (['yaw'], 'greetings'), (['wassup'], 'greetings'), (['hi'], 'greetings'), (['hey'], 'greetings'), (['holla'], 'greetings'), (['hello'], 'greetings'), (['bye'], 'goodbye'), (['good', 'bye'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['Thanks'], 'thanks'), (['okay'], 'thanks'), (['Thank', 'you'], 'thanks'), (['thankyou'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['wow'], 'thanks'), (['great'], 'thanks'), ([], 'noanswer'), (['what', "'s", 'your', 'name', '?'], 'name1'), (['who', 'are', 'you', '?'], 'name1'), (['my', 'name', 'is'], 'name'), (['I', "'m"], 'name'), (['I', 'am'], 'name'), (['coffee', '?'], 'date'), (['can', 'i', 'take', 'you', 'out', 'on', 'a', 'date'], 'date'), (['I', 'need', 'a', 'favour'], 'fav'), (['can', 'you', 'help', 'me'], 'fav'), (['I', 'need', 'you'], 'need'), (['

In [20]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [21]:
# training initializer
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])


In [27]:
random.shuffle(training)

In [28]:
# Separate bag-of-words representations and output labels
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]

In [31]:
# Convert to NumPy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

In [34]:
len(train_y[0])

55

In [32]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               16256     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 55)                3575      
                                                                 
Total params: 28087 (109.71 KB)
Trainable params: 28087 (109.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [36]:
# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
20/20 [==============================] - 1s 3ms/step - loss: 4.0039 - accuracy: 0.0300
Epoch 2/200
20/20 [==============================] - 0s 5ms/step - loss: 3.9285 - accuracy: 0.1000
Epoch 3/200
20/20 [==============================] - 0s 3ms/step - loss: 3.8831 - accuracy: 0.0900
Epoch 4/200
20/20 [==============================] - 0s 2ms/step - loss: 3.7580 - accuracy: 0.1600
Epoch 5/200
20/20 [==============================] - 0s 1ms/step - loss: 3.6767 - accuracy: 0.1500
Epoch 6/200
20/20 [==============================] - 0s 2ms/step - loss: 3.6121 - accuracy: 0.1400
Epoch 7/200
20/20 [==============================] - 0s 1ms/step - loss: 3.5153 - accuracy: 0.2000
Epoch 8/200
20/20 [==============================] - 0s 1ms/step - loss: 3.4484 - accuracy: 0.2000
Epoch 9/200
20/20 [==============================] - 0s 1ms/step - loss: 3.3121 - accuracy: 0.2100
Epoch 10/200
20/20 [==============================] - 0s 1ms/step - loss: 3.2762 - accuracy: 0.2000
Epoch 11/

In [37]:
model.save("chatbot_model.h5", hist)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
